In [3]:
import torch
import pandas as pd
import numpy as np

In [4]:
# ID processing
# ID processing
ids =  open("interpro_test_term_ids.txt", "r")
lines = ids.read().split()
test_id_dict = {}
reverse_test_id_dict = {}
for i in range(0,len(lines),2):
    # print(lines[i])
    # print(lines[i+1])
    test_id_dict[i/2] = lines[i]
    reverse_test_id_dict[lines[i]] = [i/2]
ids.close()


ids =  open("interpro_term_ids.txt", "r")
lines = ids.read().split()
train_id_dict = {}
reverse_train_id_dict = {}
for i in range(0,len(lines),2):
    # print(lines[i])
    # print(lines[i+1])
    train_id_dict[i/2] = lines[i]
    reverse_train_id_dict[lines[i]] = [i/2]
ids.close()


In [5]:
# ID processing
ids =  open("interpro_test_term_ids.txt", "r")
lines = ids.read().split()
id_dict = {}
for i in range(0,len(lines),2):
    # print(lines[i])
    # print(lines[i+1])
    id_dict[i/2] = lines[i]
    

In [6]:
# train Term Processing 
terms = open("interpro_test_terms.txt", "r")
lines = terms.read().split('\n')
test_term_dict = {}
test_num_categories = 1
saved = ""
for i in range(len(lines)):
    line_array = lines[i].split()
    test_term_dict[line_array[0]] = list(map(int, line_array[1:]))
    # if num_categories < max(term_dict[line_array[0]]):
    #     saved = line_array
    test_num_categories = max(test_num_categories, max(test_term_dict[line_array[0]]))
    
#     if i > 10:
#         break
# print(term_dict)
test_num_categories = test_num_categories+1
print("num train term categories:")
print(test_num_categories)
# print(saved)


num train term categories:
27190


In [7]:
# Train Term Processing
terms = open("interpro_terms.txt", "r")
lines = terms.read().split('\n')
train_term_dict = {}
train_num_categories = 1
saved = ""
for i in range(len(lines)):
    line_array = lines[i].split()
    train_term_dict[line_array[0]] = list(map(int, line_array[1:]))
    # if num_categories < max(term_dict[line_array[0]]):
    #     saved = line_array
    train_num_categories = max(train_num_categories, max(train_term_dict[line_array[0]]))
    
#     if i > 10:
#         break
# print(term_dict)
train_num_categories = train_num_categories+1
print(train_num_categories)
# print(saved)

27612


In [8]:
# Create unified Interpro labels
# test_id_dict
# keys: numbers (int)
# values: interpro class IDs (string) 

# test_term_dict
# keys: Uniprot ids (string)
# Values : list of Ids (list of int)

# reverse_test_id_dict
# keys: interpro class IDs (string)
# values: numbers (int)

# train_id_dict
# train_term_dict

# 1. Let train_id_dict be the main dict
main_id_dict = train_id_dict
# 2. For id in test_id_dict, create a dict that maps from the old value to the new:
# e.g. old id for a class is 3 -> 27 bc 27 is the id for that class in train_id_dict
old_test_to_new_test = {}
count = 0
for key in test_id_dict.keys():
    cur_ID = test_id_dict[key]  
    try:
        # str
        cur_ID = test_id_dict[key]
        # new number
        new_num = reverse_train_id_dict[cur_ID]
        old_test_to_new_test[key] = int(new_num[0])
        count = count+1
    except:
        new_num = len(train_id_dict)
        reverse_train_id_dict[cur_ID] = new_num
        train_id_dict[new_num] = cur_ID
        old_test_to_new_test[key] = new_num

print(count)
num_categories = len(train_id_dict)+1
print(num_categories)
# If ID is not contained within dict, create new and assign it the last number
# 3. Reassign the ids numbers in test_term_dict
new_test_term_dict = {}
for key in test_term_dict.keys():
    cur_list = test_term_dict[key]
    new_list = []
    for num in cur_list:
        new_list.append(old_test_to_new_test[num])
    new_test_term_dict[key] = new_list
print(len(new_test_term_dict))
print(cur_list)
print(new_list)
print(old_test_to_new_test[cur_list[0]])
test_term_dict = new_test_term_dict

24543
30260
135378
[276, 270]
[828, 821]
828


In [9]:
test_term_dict = new_test_term_dict

In [10]:
print( len(train_id_dict)+1)

30260


In [11]:
# Encode test term dicts
print(len(test_term_dict))
print(num_categories)
test_multi_hot_labels = torch.zeros((len(test_term_dict),num_categories),dtype=torch.float32)
count = 0
for prot in test_term_dict:
    # temp = torch.zeros(num_categories)
    int_terms = test_term_dict[prot]
    # temp[int_terms] = 1
    test_multi_hot_labels[count][int_terms] = 1
    # multi_hot_labels = torch.cat((multi_hot_labels,temp),1)
    count += 1
print(test_multi_hot_labels.shape)
print(torch.sum(test_multi_hot_labels))


135378
30260
torch.Size([135378, 30260])
tensor(545842.)


In [12]:
# Encode test and train term dicts
print(len(train_term_dict))
print(num_categories)
train_multi_hot_labels = torch.zeros((len(train_term_dict),num_categories),dtype=torch.float32)
count = 0
for prot in train_term_dict:
    # temp = torch.zeros(num_categories)
    int_terms = train_term_dict[prot]
    # temp[int_terms] = 1
    train_multi_hot_labels[count][int_terms] = 1
    # multi_hot_labels = torch.cat((multi_hot_labels,temp),1)
    count += 1
print(train_multi_hot_labels.shape)
print(torch.sum(train_multi_hot_labels))


127218
30260
torch.Size([127218, 30260])
tensor(541716.)


In [13]:
combined_ten = torch.cat((test_multi_hot_labels,train_multi_hot_labels),dim=0)
print("combined tensor shape:")
print(combined_ten.shape)

RuntimeError: [enforce fail at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 31784619840 bytes.

In [14]:
from sklearn.decomposition import NMF, PCA, TruncatedSVD
# model = NMF(n_components=1024, init= 'random', random_state=0)
# W = model.fit_transform(multi_hot_labels)
svd = TruncatedSVD(n_components=1024)
svd.fit(train_multi_hot_labels)

MemoryError: Unable to allocate 14.3 GiB for an array with shape (127218, 30260) and data type float32

In [ ]:
print(svd.transform(train_multi_hot_labels))


In [ ]:
from sklearn.decomposition import NMF, PCA, TruncatedSVD
# model = NMF(n_components=1024, init= 'random', random_state=0)
# W = model.fit_transform(multi_hot_labels)
svd = TruncatedSVD(n_components=1024)
svd.fit(combined_ten)

NameError: name 'combined_ten' is not defined

In [ ]:
# from sklearn.decomposition import NMF, PCA, TruncatedSVD
# # model = NMF(n_components=1024, init= 'random', random_state=0)
# # W = model.fit_transform(multi_hot_labels)
# svd_trans = TruncatedSVD(n_components=2)
# svd_trans.fit(torch.transpose(multi_hot_labels,0,1))

TruncatedSVD()

In [ ]:
# print(np.shape(svd.components_))
# print(np.shape(left_sing_vect))
# print(np.shape(svd_trans.components_))
# print(np.shape(svd.transform(multi_hot_labels)))
# print(left_sing_vect)
test_trans_Interpro = svd.transform(test_multi_hot_labels)
np.save("test_embeds.npy",test_trans_Interpro)
test_ids = [*test_term_dict]
np.save("test_ids.npy"mtest_ids)
train_trans_Interpro = svd.transform(train_multi_hot_labels)
np.save("train_embeds.npy",train_trans_Interpro)
train_ids = [*train_term_dict]
np.save("train_ids.npy", train_ids)
# print(svd.n_features_in_)

[[ 4.4540856e-03  1.3745555e-02]
 [ 7.6809247e-06  3.2748242e-06]
 [ 4.2494116e-06  4.4854132e-06]
 ...
 [ 1.8362008e-03  1.0325168e-03]
 [ 9.5295710e-22  2.8189123e-20]
 [ 2.2389402e-04 -5.9745670e-04]]
tensor([[ 4.4541e-03,  1.3746e-02],
        [ 7.6809e-06,  3.2748e-06],
        [ 4.2494e-06,  4.4854e-06],
        ...,
        [ 1.8362e-03,  1.0325e-03],
        [ 9.5296e-22,  2.8189e-20],
        [ 2.2389e-04, -5.9746e-04]])


In [ ]:
lit = test_term_dict
print([*lit])

['A0A023PXF5', 'A0A024RBG1', 'A0A060D764', 'A0A061ACU2', 'A0A067Z9B6', 'A0A075B6H7', 'A0A075B6H8', 'A0A075B6H9', 'A0A075B6I0', 'A0A075B6I1', 'A0A075B6I3', 'A0A075B6I4', 'A0A075B6I6', 'A0A075B6I7', 'A0A075B6I9', 'A0A075B6J1', 'A0A075B6J2', 'A0A075B6J6', 'A0A075B6J9', 'A0A075B6K0', 'A0A075B6K2', 'A0A075B6K4', 'A0A075B6K5', 'A0A075B6K6', 'A0A075B6L2', 'A0A075B6L6', 'A0A075B6N1', 'A0A075B6N2', 'A0A075B6N3', 'A0A075B6N4', 'A0A075B6P5', 'A0A075B6Q5', 'A0A075B6R0', 'A0A075B6R2', 'A0A075B6R9', 'A0A075B6S2', 'A0A075B6S4', 'A0A075B6S5', 'A0A075B6S6', 'A0A075B6S9', 'A0A075B6T6', 'A0A075B6T7', 'A0A075B6T8', 'A0A075B6U4', 'A0A075B6V5', 'A0A075B6W5', 'A0A075B6X5', 'A0A075B734', 'A0A075B759', 'A0A075B767', 'A0A077DF94', 'A0A078BQN7', 'A0A078BQP2', 'A0A087WPF7', 'A0A087WSX0', 'A0A087WSY4', 'A0A087WSY6', 'A0A087WSZ0', 'A0A087WSZ9', 'A0A087WT01', 'A0A087WT02', 'A0A087WT03', 'A0A087WTH1', 'A0A087WTH5', 'A0A087WUL8', 'A0A087WV53', 'A0A087WV62', 'A0A087WVF3', 'A0A087WW87', 'A0A087WXM9', 'A0A087WXS9', 'A0A0